<a href="https://colab.research.google.com/github/davidnoone/GEOPHYS_NOTEBOOKS/blob/main/StreamFunction_periodic_FFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Poisson equation using Fourier method

The basic problem is to solve

$$
\zeta = \nabla^2 \psi 
       =  \frac{\partial^2 \psi}{\partial x^2} 
        + \frac{\partial^2 \psi}{\partial y^2}
$$

Let us write note the discrete fourier transform of some quantity (e.g., streamunction) is epressed as the summation of Fourier series such that:

$$
\psi(x,y) \approx \frac{1}{N}\sum^{K-1}_{k=0} \hat{\psi}_k(y)  e^{ i k x }
$$


The wave number $k$ is properly defined as

$$
k = \frac{2 \pi}{L}
$$
Where L is the length of the domain. 


The (discrete) Fourier transform needed to obtain the (complex) coefficients is:
$$
\hat{\zeta}_k(y) \approx  \sum^{N-1}_{n=0}\zeta (x,y)  e^{ -i k x }
$$

N is the number of data points, at discrete positions n corresponding to values of x. One can note that with the symmetry condition, one can perform the sumation over the range $N/2$, and multiply by 2 to obtain the same result, as is done in practice below. There is a maximum of $K=N/2$ unique coefficients.

Since the transfom is linear, one can consider each harmonic, k. The  second derivative can be set analytically.

$$
\frac{\partial \psi_k(x,y)}{\partial x} = 
   \frac{\partial}{\partial x}  \left(  \hat{\psi}_k(y)  e^{ i k x } \right) =
   i k \hat{\psi}_k(y)
$$


And following, 

$$
\frac{\partial^2 \psi_k(x,y)}{\partial x^2} = -k^2 \hat{\psi}_k(y)
$$

Returning to the Laplacian, for wave number k, the "x" component follows directly as above, and we can write 


$$
\zeta_k(x,y) = \frac{\partial^2 \hat{\psi_k}}{\partial x^2} + \frac{\partial^2 \hat{\psi_k}}{\partial y^2}
$$

where $\hat{\zeta_k}$ are Fourier coefficients from the transform similar to the above. So, 

$$
\hat{\zeta_k}(y) = -k^2 \hat{\psi_k} + \frac{\partial^2 \hat{\psi_k}}{\partial y^2}
$$

Where we have used the fact that for a singl harmonic, the basis function $e^{ikx}$ is common, and can be cancelled on both sides.

Let us now apply finite differences in the "y" direction to give. 

$$
\hat{\zeta}_{n,j} = -k^2 \hat{\psi}_{k,j} + 
   \frac{1}{\Delta y^2} 
     \left(\hat{\psi}_{k,j-1} - 2 \hat{\psi}_{k,j} + \hat{\psi}_{k,j+1} \right)
$$

With this formulation, and the use of 2 boundary conditions at j=0, and j=NY, the system of "NY" equations can be seen formulated as a tridiagonal matrix.

$$
Z = A P
$$

With matrix $A$ composed of upper, central and lower diagonals as:

$$
a_i = \frac{1}{\Delta y^2}
$$
with $a_1$ = 0.

$$
b_i = \frac{1}{\Delta y^2}
$$

$$
c_i = -k^2 -\frac{2}{\Delta y^2}
$$
with $c_{M}$ = 0.

With the constraint vector Z containing values for the (complex) vorticity coefficients, matrix $A$ may be inverted using the Thompson algorithm, and with back substitution used to find the vector of streamfunction coefficients $P$, that contains values $\hat{\psi}_k(y)$. 

Finally, from $\hat{\psi}_k(y)$, the result is found by the inverse Fourier transform to provide $\psi(x,y)$ as needed.




In [ ]:
#!/bin/env python
#
# Example of how to solve the Poisson equation in 2d, using FFT in "x" and finite differences in "Y"
#
#%%
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import rfft2, irfft2

import time

# Define the grid parameters
nx = 36
ny = nx//2 + 1                  # make height half the width, but with "poles"
Lx = 1.0
Ly = 0.5

dx = Lx / nx
dy = Ly / (ny-1)                # includes end points ("poles")
print('dx=',dx,'  dy=',dy)

# Gir set up (handy for plotting.)
x = np.linspace(0, Lx, nx)
y = np.linspace(0, Ly, ny, endpoint=True)
X, Y = np.meshgrid(x, y)

# Define the vorticity  
vor = np.zeros((ny, nx))        # use index conventions similar to "nlat, nlon"
vor[2*ny//4:3*ny//4, nx//4:2*nx//4] = 1.0 

#vor = 2*np.sin(2*np.pi*X/Lx)*np.sin(np.pi*Y/Ly)


# Plot the Vorticity
#%
fig, ax = plt.subplots()
cs = ax.contourf(X, Y, vor, cmap='Spectral')
ax.set_title('Vorticity')
plt.colorbar(cs)
ax.set_aspect('equal')
plt.show()


In [ ]:
#
# JACOBI METHOD: does full field 
#  (not successive, not red/black, not over relaxation)
#

# set up indices allowing for for i-1 and i+1 points (these become global variables)
ic = np.arange(nx)
im = np.roll(ic,+1)   # weap at low end
ip = np.roll(ic,-1)   # wrap at high end

def jacobi(psi, vor, dx, dy):
    """ Jacobi iteratin to update streamfunction estimate. 
    Allow dx and dy to be different"""

    rdxsq = 1./(dx*dx)
    rdysq = 1./(dy*dy)
    rden = 0.5/(rdxsq + rdysq)
    # Create a copy of psi to store the updated values
    psi_new = psi.copy()

    for j in range(1, ny - 1):    # exclude ends/boundary values
        jc = np.repeat(j,nx)
        psi_new[jc, ic] = rden*( rdxsq*(psi[jc,   im] + psi[jc,   ip]) + \
                                 rdysq*(psi[jc-1, ic] + psi[jc+1, ic]) \
                                  - vor[jc, ic] )
    return psi_new

def solve_poisson_jacobi(vor,dx,dy):

  # Define the convergence criteria, and iterate
  maxiter = 10000
  max_error = 1e-6

  psi = np.zeros_like(vor)  # first guess
  for iteration in range(maxiter):
      # Compute the new value of psi: requires dx=dy!
      psi_new = jacobi(psi, vor, dx, dy)

      # Check for convergence
      error = np.max(np.abs(psi_new - psi))
      if error < max_error:
          break

      # Update psi with the new values
      psi = psi_new

  print('Converged after ',iteration,' iterations')
  return psi

# Do it!
t0 = time.process_time()
psi = solve_poisson_jacobi(vor,dx,dy)
delt = time.process_time()-t0
print('Jacobi solver in',delt*1000,'msec')

# Plot the solution
#%
fig, ax = plt.subplots()
cs = ax.contourf(X, Y, psi, cmap='Spectral')
ax.set_title('Streamfunction (Jacobi, T='+str(int(delt*1000))+' msec)')
plt.colorbar(cs)
ax.set_aspect('equal')
plt.show()

In [ ]:
def tridiagonal_solve(a, b, c, d):
    """
    Solve a tridiagonal system Ax = d using Thompson's algorithm.
       a[i]*x[i-1] + b[i]*x[i] + c[i]*x[i+1] = d[i]

       a[0] and c[nx] are not used. 
    
    Parameters:
        a (ndarray): Lower diagonal of A. [FLOAT]
        b (ndarray): Main diagonal of A. [FLOAT]
        c (ndarray): Upper diagonal of A. [FLOAT]
        d (ndarray): Right-hand side vector. [FLOAT or COMPLEX]
        
    Returns:
        x (ndarray): Solution vector. [type same as "d"]
    """
      
    # Initialize variables
    n = len(b)
    x = np.zeros_like(d)
    cloc = c.copy()       # work variable
    dloc = d.copy()       # work variable
 
    # Forward elimination
    cloc[0] /= b[0]
    dloc[0] /= b[0]
    for i in range(1, n):
        temp = b[i] - a[i] * cloc[i-1]
        cloc[i] /= temp
        dloc[i] = (dloc[i] - a[i] * dloc[i-1]) / temp
        
    # Backward substitution
    x[n-1] = dloc[n-1]
    for i in range(n-2, -1, -1):
        x[i] = dloc[i] - cloc[i] * x[i+1]
        
    return x

In [ ]:
#
# FFT METHOD in "x", then use finite differences in "y", which
# has a direct solution using a tridiagonal solver.
#
# number of unique wave numbers (other half is symmetric)
def solve_poisson_fft(vor,dx,dy):
    nfour = nx//2 + 1   
    rdysq = 1./(dy*dy)

    # Compute the Fourier coefficients of the vorticity
    vor_hat = rfft2(vor, axes=[1])

    # Compute the wavenumbers
    kx = 2*np.pi * np.fft.rfftfreq(nx, d=dx)  # can be nfour?
    kx[0] = 1.                                # avoid div by zero


    # Set off diagonal matrix coefficients
    a =  np.ones(ny)*rdysq
    c =  np.ones(ny)*rdysq
    a[-1] = 0.0
    c[0] = 0.0


    # Solve per wave number: m=0 carries boundary conditions (psi=0).
    psi_hat = np.zeros_like(vor_hat)
    for m in range(0,nfour):
        b = -(a + c) - kx[m]**2
        psi_hat[:,m] = tridiagonal_solve(a, b, c, vor_hat[:,m])      

    # Transform the solution back to physical space
    psi = np.fft.irfft2(psi_hat, axes=[1])

    return psi

# Do it!
t0 = time.process_time()
psi = solve_poisson_fft(vor,dx,dy)
delt = time.process_time()-t0
print('FFT solver in',delt*1000,'msec')

# Plot the solution
#%
fig, ax = plt.subplots()
cs = ax.contourf(X, Y, psi, cmap='Spectral')
ax.set_title('Streamfunction (FFT, T='+str(int(delt*1000))+' msec)')
plt.colorbar(cs)
ax.set_aspect('equal')
plt.show()

